In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from scipy import sparse
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
%cd '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation'
%ls

/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation
data/      Metric.py  __pycache__/  Result.py  test.ipynb
Loader.py  model/     README.md     Run.py     Untitled0.ipynb


In [5]:
import Loader as ld
import Metric as me
import Result as rs

In [6]:
%cd '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF'
%ls

/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF
best_model/  MF.py  __pycache__/


In [7]:
import MF as mf

# 1. 학습
## 1-1. 데이터 로드

In [8]:
DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/preprocessed/'
loader1 = ld.Loader(DIR, 1, 8)
loader1.load_dataset()

(24032, 2661)

In [ ]:
train = loader1.formatting(0)

In [9]:
valid_pos, valid  = loader1.formatting(1)
valid_y, valid_X = valid
test_pos, test  = loader1.formatting(2)
test_y, test_X = test

In [ ]:
train_usr, train_prb, train_entry = shuffle(train[0], train[1], train[2])
train_usr = np.array(train_usr).reshape(-1,1)
train_prb = np.array(train_prb).reshape(-1,1)
train_entry = np.array(train_entry).reshape(-1,1)

##1-2. 모델 학습

In [10]:
mf = mf.MF(loader1.users_no, loader1.prob_no)
model = mf.get_model()

In [11]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='mse')

In [12]:
train_N = len(train_usr)-1
valid_N = len(valid_X)-1
epochs = 1
batch_size = 4096
rc_vad = []
hr_vad = []
best_eval = -1
best_epoch = -1

MODEL_DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF/best_model'

In [13]:
model = tf.keras.models.load_model(MODEL_DIR)

In [82]:
for epoch in range(epochs):
  '''for i in range(0, train_N, batch_size):
    idxlist = range(i, min(i+batch_size, train_N))
    model.fit([train_usr[idxlist], train_prb[idxlist]], train_entry[idxlist],verbose=0)'''
  
  hit_rate = 0
  recall = []
  for i in range(0, valid_N, batch_size):
    idxlist = range(i, min(i+batch_size, valid_N))
    valid_batch = valid_X.iloc[loader1.get_idx(valid_X, idxlist)]
    if len(valid_batch) <= 0:
      continue

    valid_X_usr = np.array(valid_batch['handle'].tolist()).reshape(-1,1)
    valid_X_prb = np.array(valid_batch['problemId'].tolist()).reshape(-1,1)

    X_pred = model.predict([valid_X_usr, valid_X_prb])

    valid_batch['pred'] = X_pred
    filtered = mf.level_filtering(valid_batch, loader1.user2level, loader1.prob2level, 30)
    pred = sparse.csr_matrix((filtered['pred'], \
                             (filtered['handle'], filtered['problemId'])),\
                             dtype='float64', shape=(loader1.users_no, loader1.prob_no)).toarray()

    valid_y_batch = valid_y.iloc[loader1.get_idx(valid_y, valid_batch['handle'].tolist())]
    heldout = sparse.csr_matrix((np.ones_like(valid_y_batch['handle']), \
                             (valid_y_batch['handle'], valid_y_batch['problemId'])),\
                             dtype='float64', shape=(loader1.users_no, loader1.prob_no)).toarray()

    recall.append(me.recall_at_k(pred, heldout, k=30))
    hit_rate += me.hit_rate_at_k(pred, heldout, k=30)

  recall_ = np.concatenate(recall)
  recall_[np.isnan(recall_)]=0 
  recall_ = recall_.mean()
  rc_vad.append(recall_)
  print("epoch[", epoch, "] recall: ", recall_)

  hit_rate_ = hit_rate/loader1.users_no
  hr_vad.append(hit_rate_)
  print("epoch[", epoch, "] hit rate: ", hit_rate_)

  if hit_rate_ > best_eval:
    model.save(MODEL_DIR)
    best_epoch = epoch
    best_eval = hit_rate_
    print(epoch, best_eval)

0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/Metric.py:10: RuntimeWarning: invalid value encountered in true_divide
  recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))


KeyboardInterrupt: ignored

In [83]:
  recall_ = np.concatenate(recall)
  recall_[np.isnan(recall_)]=0 
  recall_ = recall_.mean()
  rc_vad.append(recall_)
  print("epoch[", epoch, "] recall: ", recall_)

  hit_rate_ = hit_rate/loader1.users_no
  hr_vad.append(hit_rate_)
  print("epoch[", epoch, "] hit rate: ", hit_rate_)

epoch[ 0 ] recall:  0.0002215685955484796
epoch[ 0 ] hit rate:  0.009570572569906791


# 1-3. test

In [ ]:
test_N = len(test_X)-1
hit_rate = 0
recall = []

for i in range(0, test_N, batch_size):
  idxlist = range(i, min(i+batch_size, valid_N))
  test_batch = test_X.iloc[loader1.get_idx(test_X, idxlist)]
  if len(test_batch) <= 0:
    continue

  test_X_usr = np.array(test_batch['handle'].tolist()).reshape(-1,1)
  test_X_prb = np.array(test_batch['problemId'].tolist()).reshape(-1,1)

  X_pred = model.predict([test_X_usr, test_X_prb])
  test_batch['pred'] = X_pred
  filtered = mf.level_filtering(test_batch, loader1.user2level, loader1.prob2level, 30)

  pred = sparse.csr_matrix((filtered['pred'], \
                             (filtered['handle'], filtered['problemId'])),\
                             dtype='float64', shape=(loader1.users_no, loader1.prob_no)).toarray()

  test_y_batch = test_y.iloc[loader1.get_idx(test_y, test_batch['handle'].tolist())]

  heldout = sparse.csr_matrix((np.ones_like(test_y_batch['handle']), \
                             (test_y_batch['handle'], test_y_batch['problemId'])),\
                             dtype='float64', shape=(loader1.users_no, loader1.prob_no)).toarray()

  recall.append(me.recall_at_k(pred, heldout, k=30))
  hit_rate += me.hit_rate_at_k(pred, heldout, k=30)

recall_ = np.concatenate(recall)
recall_[np.isnan(recall_)]=0 
recall_ = recall_.mean()
rc_vad.append(recall_)
print("test recall: ", recall_)

hit_rate_ = hit_rate/loader1.users_no
hr_vad.append(hit_rate_)
print("test hit rate: ", hit_rate_)

# 2. 결과

In [80]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import ast

class Result:
    def __init__(self, MODEL_DIR, DATASET_DIR, data_no, batch_size):
        self.model = tf.keras.models.load_model(MODEL_DIR)
        
        users = pd.read_csv(os.path.join(DATASET_DIR+'/preprocessed/data1', 'd'+data_no+'_users.csv'))
        problems =  pd.read_csv(os.path.join(DATASET_DIR+'/preprocessed/data1', 'd'+data_no+'_problems.csv'))
        solvedProblem = pd.read_csv(os.path.join(DATASET_DIR+'/raw_data', 'solvedProblem.csv'))

        self.N = len(problems['problemId'])
        self.id2idx_usr = {u[0]:i for i, u in enumerate(users.values)}
        self.id2prblist = {u[1]:u[3] for u in solvedProblem.values}
        self.idx2id_prb = {i:p[0] for i, p in enumerate(problems.values)}
        
        self.batch_size = batch_size
  
    def get_result(self, id):
        entry = []
        probs = list(set(range(0, self.N))-set(ast.literal_eval(self.id2prblist[id])))
        for i in range(0, self.N, self.batch_size):
            idxlist = probs[i:min(i+self.batch_size, self.N)]

            input_p = np.array(idxlist).reshape(-1, 1)
            input_u = np.array([self.id2idx_usr[id]]*len(idxlist)).reshape(-1, 1)

            entry.append(self.model.predict([input_u, input_p]))

        entry = np.concatenate(entry)
        entry = np.array(entry).reshape(1, -1)
        top_idx = np.argpartition(-entry, 30, axis=1)
        return [self.idx2id_prb[i] for i in top_idx[0]]

In [81]:
MODEL_DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF/best_model'
DATA_DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data'
result = Result(MODEL_DIR, DATA_DIR, "1", 1024)
print(result.get_result("beoms"))

[19698, 9536, 15500, 2410, 1612, 2720, 22150, 14426, 1544, 14038, 9455, 11387, 19946, 1890, 6550, 9517, 15885, 2529, 14492, 10280, 9947, 20268, 2525, 21633, 9934, 2476, 4176, 20651, 20152, 20363, 4084, 1654, 1100, 14625, 16017, 1924, 24445, 4883, 15876, 15656, 23841, 1620, 13411, 2845, 1182, 18512, 10039, 15964, 6811, 9664, 1302, 15786, 6588, 16506, 15115, 23334, 10929, 4411, 1977, 15792, 4806, 14584, 6122, 10102, 16114, 10695, 16485, 9184, 17177, 10774, 21869, 11654, 16170, 17103, 11720, 17262, 23842, 4118, 14173, 13224, 9550, 9322, 4892, 16462, 10940, 8393, 24508, 23802, 6794, 11448, 24267, 9657, 5554, 5598, 23881, 10497, 22864, 18156, 2512, 16426, 2082, 10815, 8815, 18063, 10406, 4659, 23738, 9524, 12836, 18127, 4706, 15787, 20920, 13063, 15761, 15751, 12330, 21156, 5074, 4335, 18826, 1331, 21235, 15858, 1439, 5176, 15728, 13240, 17266, 9612, 13410, 20495, 13416, 3182, 13610, 21608, 3060, 3058, 3010, 5338, 17284, 1303, 18243, 10426, 11726, 11655, 18290, 21866, 1535, 5671, 14231, 203

# 3. 난이도 필터링

In [ ]:
user_level = {i:row[1] for i, row in enumerate(loader1.users.values)}
prob_level = {i:row[1] for i, row in enumerate(loader1.problems.values)}

In [ ]:
def level_filtering(self, dataframe, userlevel_map, problevel_map, k):
  
        user = dataframe['handle'].to_numpy()
        prob = dataframe['problemId'].to_numpy()
        pred = dataframe['pred'].to_numpy()
        
        limit = min(k*4, len(pred))
        idx = np.argpartition(-pred, limit)[:limit]
        
        candidates = dataframe.iloc[idx]
        problevel = candidates['problemId'].apply(lambda x: problevel_map[x]).to_numpy()
        maxlevel = candidates['handle'].apply(lambda x: userlevel_map[x]).to_numpy()
        dist = np.abs(problevel-maxlevel)
          
        top_idx = np.argpartition(dist, k)[:limit]
        top_k_data = dataframe.iloc[top_idx]
        
        return top_k_data

In [ ]:
for i in loader1.train['handle'].values:
  print(i)
  break

In [ ]:
train_N = len(train_usr)-1
valid_N = len(valid_X)-1
epochs = 1
batch_size = 1024
rc_vad = []
hr_vad = []
best_eval = -1
best_epoch = -1

MODEL_DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF/best_model'
model = tf.keras.models.load_model(MODEL_DIR)

In [ ]:
for epoch in range(epochs):
  hit_rate = 0
  recall = []
  for i in range(0, valid_N, batch_size):
    print(i)
    idxlist = range(i, min(i+batch_size, valid_N))
    valid_batch = valid_X.iloc[loader1.get_idx(valid_X, idxlist)]
    if len(valid_batch) <= 0:
      continue

    valid_X_usr = np.array(valid_batch['handle'].tolist()).reshape(-1,1)
    valid_X_prb = np.array(valid_batch['problemId'].tolist()).reshape(-1,1)

    X_pred = model.predict([valid_X_usr, valid_X_prb])
    valid_batch['pred'] = X_pred
    filtered = level_filtering(valid_batch, user_level, prob_level, 30)

    pred = sparse.csr_matrix((filtered['pred'], \
                             (filtered['handle'], filtered['problemId'])),\
                             dtype='float64', shape=(loader1.users_no, loader1.prob_no)).toarray()

    valid_y_batch = valid_y[0].iloc[loader1.get_idx(valid_y[0], valid_batch['handle'].tolist())]
    heldout = sparse.csr_matrix((np.ones_like(valid_y_batch['handle']), \
                             (valid_y_batch['handle'], valid_y_batch['problemId'])),\
                             dtype='float64', shape=(loader1.users_no, loader1.prob_no)).toarray()

    recall.append(me.recall_at_k(pred, heldout, k=30))
    hit_rate += me.hit_rate_at_k(pred, heldout, k=30)

  recall_ = np.concatenate(recall)
  recall_[np.isnan(recall_)]=0 
  recall_ = recall_.mean()
  rc_vad.append(recall_)
  print(epoch, recall_)

  hit_rate_ = hit_rate/loader1.users_no
  hr_vad.append(hit_rate_)
  print(epoch, hit_rate_)

  if hit_rate_ > best_eval:
    model.save(MODEL_DIR)
    best_epoch = epoch
    best_eval = hit_rate_
    print(epoch, best_eval)

In [ ]:
prob_level[2993]